In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

## Check clip sizes

In [3]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'dev') 

splits=C.split_files()

L=[x['t_seconds'] for x in splits]

min(L),max(L),sum(L)/len(L)

len(splits)

## Add vocoder clips to training set

In [9]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'build') 
splits=C.split_files()

In [10]:
splits[0]

{'name': 'NIST/openasr20_amharic/build/audio_split/BABEL_OP3_307_71263_20140602_180728_outLine_3079760_3092720.wav',
 'key': ['BABEL',
  'OP3',
  '307',
  '71263',
  '20140602',
  '180728',
  'outLine',
  '3079760',
  '3092720'],
 'channel': 'outLine',
 'start': 3079760,
 'end': 3092720,
 't_seconds': 0.81,
 't_begin': 192.485}

In [11]:
import sys
sys.path.append('/home/catskills/Desktop/openasr/Python-WORLD')

from pathlib import Path
import os
import numpy as np
from scipy.io.wavfile import read as wavread
from scipy.io.wavfile import write as wavwrite
from scipy import signal
import librosa
from world import main

ModuleNotFoundError: No module named 'world'

In [ ]:
wav_path = splits[0]['name']


os.getcwd()



x,fs=librosa.load(wav_path)

fs

if 0:  # resample
    fs_new = 16000
    x = signal.resample_poly(x, fs_new, fs)
    fs = fs_new

if 0:  # low-cut
    B = signal.firwin(127, [0.01], pass_zero=False)
    A = np.array([1.0])
    if 0:
        import matplotlib.pyplot as plt
        w, H = signal.freqz(B, A)

        fig, (ax1, ax2) = plt.subplots(2, figsize=(16, 6))
        ax1.plot(w / np.pi, abs(H))
        ax1.set_ylabel('magnitude')
        ax2.plot(w / np.pi, np.unwrap(np.angle(H)))
        ax2.set_ylabel('unwrapped phase')
        plt.show()
    x = signal.lfilter(B, A, x)

vocoder = main.World()

# analysis
dat = vocoder.encode(fs, x, f0_method='harvest', is_requiem=True) # use requiem analysis and synthesis

if 0:  # global pitch scaling
    dat = vocoder.scale_pitch(dat, 1.5)
if 0:  # global duration scaling
    dat = vocoder.scale_duration(dat, 2)
if 0:  # fine-grained duration modification
    vocoder.modify_duration(dat, [1, 1.5], [0, 1, 3, -1])  # TODO: look into this


# dat['f0'] = np.r_[np.zeros(5), dat['f0'][:-5]]

for key in dat:
    try:
        print(key, dat[key].shape, dat[0:4])
    except:
        print(key, dat[key])

# synthesis
dat = vocoder.decode(dat)

for key in dat:
    try:
        print(key, dat[key].shape)
    except:
        print(key, dat[key])

cycles_per_frame=dat['out'].shape[0]/dat['vuv'].shape[0]
cycles_per_frame

frames_per_second=dat['fs']/cycles_per_frame
frames_per_second

if 0:  # audio
    import simpleaudio as sa
    snd = sa.play_buffer((dat['out'] * 2 ** 15).astype(np.int16), 1, 2, fs)
    snd.wait_done()


%matplotlib inline
import matplotlib.pylab as plt
vocoder.draw(x, dat, figure_size=(12,12))

wav_path[0:-4]

output_fn=wav_path[0:-4]+ '-resynth.wav'
wavwrite(output_fn, fs, (dat['out'] * 2 ** 15).astype(np.int16))

output_fn

import IPython
IPython.display.Audio(wav_path)

IPython.display.Audio(output_fn)

from world.get_seeds_signals import get_seeds_signals

ss=get_seeds_signals(dat['fs'])

ss['pulse'].shape

ss['noise'].shape

## Train step 1: Bootstrap from pretrained model

In [ ]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'build') 

In [ ]:
from load_pretrained_amharic_model import load_pretrained_amharic_model
model = load_pretrained_amharic_model(C, 0)

In [ ]:
import pytorch_lightning as pl
import os, datetime

model_save_dir='save/nemo_amharic'

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

trainer = pl.Trainer(gpus=[0], max_epochs=200, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

In [ ]:
from ruamel.yaml import YAML
from omegaconf import DictConfig
config_path = 'amharic_16000.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest
model.set_trainer(trainer)
model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

In [ ]:
from reshuffle_samples import reshuffle_samples
reshuffle_samples(C)

In [ ]:
trainer.fit(model)

## Apply vocoder filter to DEV

## DEV translation

In [ ]:
from Cfg import Cfg
from glob import glob
from package_DEV import package_DEV
from load_pretrained_amharic_model import load_pretrained_amharic_model
version='113'
C = Cfg('NIST', 16000, 'amharic', 'dev', version)
model = load_pretrained_amharic_model(C, 0)
files=list(sorted(glob(f'{C.audio_split_dir}/*.wav')))
translations=model.transcribe(paths2audio_files=files, batch_size=32)
